In [0]:
!pip install census
!pip install us


[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.4/347.4 kB 6.4 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [0]:
from census import Census
from us import states
from pyspark.sql.types import *

In [0]:
# Variables: https://api.census.gov/data/2020/acs/acs5/variables.html
c = Census('cc5fcc45a6ba3290f753f8a5a3a6b494551ba417', year=2020)

In [0]:
id_census = c.acs5.state_county_tract(
    fields = ('NAME', 'C17002_001E', 'C17002_002E', 'C17002_003E', 'B01003_001E'),
    state_fips = states.ID.fips,
    county_fips = "*",
    tract = "*",
    year = 2020)

In [0]:
print(id_census)

[{'NAME': 'Census Tract 9714.01, Bonneville County, Idaho', 'C17002_001E': 3547.0, 'C17002_002E': 134.0, 'C17002_003E': 217.0, 'B01003_001E': 3680.0, 'state': '16', 'county': '019', 'tract': '971401'}, {'NAME': 'Census Tract 9714.02, Bonneville County, Idaho', 'C17002_001E': 5876.0, 'C17002_002E': 48.0, 'C17002_003E': 225.0, 'B01003_001E': 5885.0, 'state': '16', 'county': '019', 'tract': '971402'}, {'NAME': 'Census Tract 9715, Bonneville County, Idaho', 'C17002_001E': 1868.0, 'C17002_002E': 29.0, 'C17002_003E': 17.0, 'B01003_001E': 1868.0, 'state': '16', 'county': '019', 'tract': '971500'}, {'NAME': 'Census Tract 9701, Boundary County, Idaho', 'C17002_001E': 7324.0, 'C17002_002E': 390.0, 'C17002_003E': 1069.0, 'B01003_001E': 7376.0, 'state': '16', 'county': '021', 'tract': '970100'}, {'NAME': 'Census Tract 9702, Boundary County, Idaho', 'C17002_001E': 4717.0, 'C17002_002E': 652.0, 'C17002_003E': 345.0, 'B01003_001E': 4780.0, 'state': '16', 'county': '021', 'tract': '970200'}, {'NAME': 

In [0]:
df = spark.createDataFrame(id_census)

# Because of the format that the census data comes over in (a list of dictionaries), it is much easier to adjust the datatypes and rename the columns after the dataframe is created
df = df.select(
    df["NAME"].alias("geo_name"),
    df["C17002_001E"].cast(DoubleType()).alias("total_poverty_count"),
    df["C17002_002E"].cast(DoubleType()).alias("income_below_poverty"),
    df["C17002_003E"].cast(DoubleType()).alias("income_1_to_1.24_poverty"),
    df["B01003_001E"].cast(DoubleType()).alias("total_population")
)

In [0]:
print(df.dtypes)

df.limit(10).display()

[('geo_name', 'string'), ('total_poverty_count', 'double'), ('income_below_poverty', 'double'), ('income_1_to_1.24_poverty', 'double'), ('total_population', 'double')]


geo_name,total_poverty_count,income_below_poverty,income_1_to_1.24_poverty,total_population
"Census Tract 9714.01, Bonneville County, Idaho",3547.0,134.0,217.0,3680.0
"Census Tract 9714.02, Bonneville County, Idaho",5876.0,48.0,225.0,5885.0
"Census Tract 9715, Bonneville County, Idaho",1868.0,29.0,17.0,1868.0
"Census Tract 9701, Boundary County, Idaho",7324.0,390.0,1069.0,7376.0
"Census Tract 9702, Boundary County, Idaho",4717.0,652.0,345.0,4780.0
"Census Tract 9701, Butte County, Idaho",2562.0,249.0,298.0,2603.0
"Census Tract 9701, Camas County, Idaho",1069.0,14.0,118.0,1069.0
"Census Tract 201, Canyon County, Idaho",1140.0,139.0,171.0,1210.0
"Census Tract 202, Canyon County, Idaho",5108.0,268.0,892.0,5108.0
"Census Tract 203, Canyon County, Idaho",4631.0,228.0,1100.0,4631.0
